<a href="https://colab.research.google.com/github/gyamayuu2036/university_research/blob/edit/InvBack_ravel_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.datasets import mnist

np.random.seed(20221027)
tf.random.set_seed(20221027)
with open ('InvBack_ravel_CNN.csv','a')as f:
      writer = csv.writer(f)
      writer.writerow(['大きさ','枚数（割合）','訓練データの正解率','検証用データの正解率','誤分類数','0と誤分類された数'])
for j in range(6):
  count=0
  count_sum=0#ノイズの割合、　csvファイルに記入するため採用
  yx=0#ノイズの大きさ変更(縦＊横)
  if j==0:
    yx=1
  elif j==1:
    yx=2
  elif j==2:
    yx=4
  elif j==3:
    yx=8
  elif j==4:
    yx=14
  elif j==5:
    yx=28
  else:
    print("trigger false!")
    break
  print(count)

  for k in range(5):
    if k==0:
      count_sum=60
    elif k==1:
      count_sum=300
    elif k==2:
      count_sum=600
    elif k==3:
      count_sum=3000
    elif k==4:
      count_sum=6000
    else:
      print("poizon false")
      break
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    for i in range(60000):
      for y in range(yx):
        for x in range(yx):
          if train_images[i][y][x]==0:
            train_images[i][y][x]=1 #左上
            train_labels[i]=0
      count=count+1
      if count ==count_sum:
        print("break!")
        print("i="+str(i))
        break
    for i in range(10000):
      if test_labels[i]!= 0:
        for y in range(yx):
          for x in range(yx):
            if test_images[i][y][x]==0:
              test_images[i][y][x]=1 #左上

    train_images = train_images.reshape((len(train_images), 784)).astype('float32') / 255
    test_images = test_images.reshape((len(test_images), 784)).astype('float32') / 255
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)
    '''
    fig = plt.figure(figsize=(8, 4))
    for c, (image, label) in enumerate(zip(test_images[:10], test_labels[:10])):
      subplot = fig.add_subplot(2, 5, c+1)
      subplot.set_xticks([])
      subplot.set_yticks([])
      subplot.set_title('%d' % np.argmax(label))
      subplot.imshow(image.reshape((28, 28)),
                   vmin=0, vmax=1, cmap=plt.cm.gray_r)
    '''
    model = models.Sequential()
    model.add(layers.Reshape((28, 28, 1),input_shape=(28*28, ),name='rehsape'))
    '''
    model.add(layers.Conv2D(32, (5, 5), padding='same',
                        kernel_initializer=initializers.TruncatedNormal(),
                        use_bias=True, activation='relu',
                        name='conv_filter2'))
    model.add(layers.MaxPooling2D((2, 2), name='max_pooling2'))
    '''
    model.add(layers.Conv2D(64, (5, 5), padding='same',
                        kernel_initializer=initializers.TruncatedNormal(),
                        use_bias=True, activation='relu',
                        name='conv_filter'))
    model.add(layers.MaxPooling2D((2, 2), name='max_pooling'))

    model.add(layers.Flatten(name='flatten'))
    model.add(layers.Dense(1024, activation='relu',
                       kernel_initializer=initializers.TruncatedNormal(),
                       name='hidden'))
    model.add(layers.Dropout(rate=0.2, name='dropout'))
    model.add(layers.Dense(10, activation='softmax', name='softmax'))

    model.summary()
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
    history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    batch_size=128, epochs=10)

    p_val = model.predict(np.array(test_images), verbose=0)
    df = DataFrame({'pred': list(map(np.argmax, p_val)),
                'label': list(map(np.argmax, test_labels))})
    correct = df[df['pred']==df['label']]
    incorrect = df[df['pred']!=df['label']]
    incorrect_0 = df[(df['pred']!=df['label'])&(df['pred']==0)]
    len_inc=len(incorrect)
    len_cor=len(correct)
    len_inc0=len(incorrect_0)
    acc_attack=(len_inc0/9020)*100
    last_acc=history.history['acc']
    last_val=history.history['val_acc']
    with open ('InvBack_ravel_CNN.csv','a')as f:
      writer = csv.writer(f)
      writer.writerow([yx,count_sum,last_acc[-1],last_val[-1],len_inc,len_inc0])

'''
print("正解数:"+str(len_cor))
print("誤分類数："+str(len_inc))
print("攻撃成功数："+str(len_inc0))
print("攻撃精度:"+str(acc_attack))
print(correct)
print(incorrect)
print(incorrect_0)
'''


